In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import random
import math
import matplotlib.pyplot as plt
import cvxopt
from sklearn.metrics import confusion_matrix, classification_report, precision_score
import time

In [2]:
n = 1000
d = 5
x = np.random.normal(size=(n,d))
error = np.random.normal(size=(n,1))

b = np.random.uniform(size=(d,1))

### in order to avoid Perfect Separation, we need to add some noise
y = x@b + error

# y = list(map(lambda x: 1 if x > 0 else 0, y))
# y = np.array(y)

# binary
y = np.array([0 if x <0 else 1  for x in y]).reshape(-1,1)
# y = np.where(y<0, 0, 1)
## 3 classes
# y = np.array([0 if x < -1 else 1 if x < 1 else 2 for x in y])


In [3]:
class data_generate_process:
    def __init__(self, x, y):
        self.x = x
        self.y = y    
        
    def split(self, rate = 0.7, random_state = 1024, scale = False):
        ## Feature scaling is used to normalize the range of independent variables or features of data
        if scale:
            self.x = (self.x - np.mean(self.x))/x.std()
        
        n = len(self.y)
        np.random.seed(random_state)
        
        ##randomly spilte data into 70% train and 30% test
        index = list(range(n))
        np.random.shuffle(index)
        train = index[:int(rate*n)]
        test = index[int(rate*n):]
        
        self.x_train = self.x[train]
        self.x_test = self.x[test]
        self.y_train = self.y[train]
        self.y_test = self.y[test]
        
        return self.x_train, self.x_test, self.y_train, self.y_test
    
x_train, x_test, y_train, y_test = data_generate_process(x, y).split()

# Support Vector Machine
## https://en.wikipedia.org/wiki/Support-vector_machine

## $min D(\beta) = 1/2 \beta' \beta $
##    $  s.t. \forall_i    y_i(x\dot \beta) \geqslant 1 $
# ====>1. Lagrangian duality
## $max D(\alpha) = \sum \alpha_i - 1/2 \sum y_i \alpha_i y_j\alpha_j \Phi(x_i, x_j)$
## $ s.t. \forall_i  \alpha_i > 0$ and $ \sum_i y_i \alpha_i = 0$
# =====> or 2. GradientDescend
## cost function $f(\beta) = 1/2 \alpha \beta ' \beta +  average (\sum_i max(0, 1 -y_i(X \beta)))$
# => Stochastic sub-Gradicent Descent
##  CostFunction $f(\beta) = 1/2 \lambda \beta ' \beta + average( C \sum_i max(0, 1 -y_i(X \beta)))$
##       $ \beta_(t+1) = \beta_t - \alpha_t (\lambda \beta_t - y_i x_i) $      if $ y_i \beta x_i <= 1 $
##            $ \beta_(t+1)= \beta_t - \alpha_t \lambda \beta_t  $ else
## LearningRate $\alpha = 1/(1 + epoch) $

In [4]:
class model:
    def __init__(self, x, y,  bias = True):
        
        if bias:
            x = self.getBias(x)
            
        self.x = x
        self.y = [1 if item > 0 else -1 for item in y]
    
    def getBias(self, x):
        ones = np.ones((len(x),1))
        return np.hstack((ones,x))
    
    def fit(self, kernel = None, parameter = 1, iteration = 500, C = 0.02, threshold = 1e-3):
        self.kernel = kernel
        self.parameter = parameter
        
        x = self.getKernel(self.x, self.x, kernel = self.kernel)
        y = self.y
        
        beta = np.zeros((x.shape[1], 1))
        
        for epoch in range(iteration):
            learning = 1/(1+epoch)
            
            for i in range(len(x)):
                if y[i]*x[i]@beta  < 1:
                    update = learning *(C*beta - (y[i]*x[i]).reshape(-1,1))
                else:
                    update = learning*(C*beta)
                
                beta -= update
                ## stop when enough close to the global minimal
                if (epoch > 1) and np.abs(update).sum()< threshold:
                    self.beta = beta
                    return self.beta
                
        self.beta = beta
        return self.beta 
    
    def getKernel(self, x, z, kernel = None):
        if self.kernel is None:
            return x
        
        if self.kernel == 'linear':
            return x@(z.T)
        
        if self.kernel == 'polynominal':
            return np.power(1 + x@z.T, self.parameter)
        
        if self.kernel == 'rbf':
            ## use euclidean_dist_matrix K(x, y) = e^(-g||x - z||^2)
            norms_1 = (x ** 2).sum(axis=1)
            norms_2 = (z ** 2).sum(axis=1)
            distance = np.abs(norms_1.reshape(-1, 1) + norms_2 - 2 * np.dot(x, z.T))
            return np.exp(-self.parameter* distance)
        
    def predict(self, x):
        x = self.getBias(x)
        newx = self.getKernel(x, self.x)
        return (newx@self.beta> 0).astype(int)


In [5]:

svm = model(x_train, y_train)
svm.fit()
pred = svm.predict(x_test)
confusion_matrix(y_test, pred)

array([[117,  40],
       [ 28, 115]], dtype=int64)

In [6]:
svm = model(x_train, y_train)
svm.fit(kernel = 'linear')
pred =svm.predict(x_test)
confusion_matrix(y_test, pred)

array([[112,  45],
       [ 39, 104]], dtype=int64)

In [7]:
svm = model(x_train, y_train)
svm.fit(kernel = 'rbf', parameter = 0.5)
pred =svm.predict(x_test)
confusion_matrix(y_test, pred)

array([[119,  38],
       [ 32, 111]], dtype=int64)

In [8]:
svm = model(x_train, y_train)
svm.fit(kernel='polynominal', parameter = 3)
pred =svm.predict(x_test)
confusion_matrix(y_test, pred)

array([[107,  50],
       [ 31, 112]], dtype=int64)

# compared to sklearn

In [9]:
from sklearn import svm
start = time.time()
linear_svc = svm.SVC(kernel='linear', gamma = 0.5)
linear_svc.fit(x_train, y_train.ravel())
y_pred = linear_svc.predict(x_test)
end = time.time()
sk_svm = end - start
confusion_matrix(y_test, y_pred)

array([[121,  36],
       [ 34, 109]], dtype=int64)

In [10]:
linear_svc = svm.SVC(kernel='rbf', gamma = 0.5)
linear_svc.fit(x_train, y_train.ravel())
y_pred = linear_svc.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[117,  40],
       [ 37, 106]], dtype=int64)

In [11]:
linear_svc = svm.SVC(kernel='poly', gamma = 0.5)
linear_svc.fit(x_train, y_train.ravel())
y_pred = linear_svc.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[111,  46],
       [ 34, 109]], dtype=int64)